In [1]:
import pandas as pd

# Load data berisi indikator dan return
df = pd.read_csv("../data/gld_clean.csv", skiprows=2)
df.columns = ['Date', 'Close', 'High', 'Low', 'Open', 'Volume']
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)
df = df[['Open', 'High', 'Low', 'Close', 'Volume']]

# Hitung indikator tambahan
df['Return'] = df['Close'].pct_change() * 100
df['Volatility'] = df['Return'].rolling(window=20).std()
df['RSI'] = 100 - (100 / (1 + df['Close'].diff().apply(lambda x: x if x > 0 else 0).rolling(14).mean() /
                         -df['Close'].diff().apply(lambda x: x if x < 0 else 0).rolling(14).mean()))
ema_12 = df['Close'].ewm(span=12, adjust=False).mean()
ema_26 = df['Close'].ewm(span=26, adjust=False).mean()
df['MACD'] = ema_12 - ema_26
df['Signal'] = df['MACD'].ewm(span=9, adjust=False).mean()

# === Statistik Deskriptif Return & Volatility ===
summary = df[['Return', 'Volatility']].describe().round(3)
print("📊 Statistik Return dan Volatilitas:\n", summary)

# === Hari Return Tertinggi & Terendah ===
top_gain = df['Return'].idxmax()
top_loss = df['Return'].idxmin()

print(f"\n🚀 Hari Return Tertinggi: {top_gain.date()} ({df.loc[top_gain, 'Return']:.2f}%)")
print(f"📉 Hari Return Terendah: {top_loss.date()} ({df.loc[top_loss, 'Return']:.2f}%)")

# === Korelasi Antar Indikator ===
correlation = df[['Return', 'Volatility', 'RSI', 'MACD', 'Signal']].corr().round(2)
print("\n🔗 Korelasi antar fitur:\n", correlation)


📊 Statistik Return dan Volatilitas:
         Return  Volatility
count  501.000     482.000
mean     0.073       0.869
std      0.896       0.233
min     -3.568       0.429
25%     -0.459       0.693
50%      0.061       0.856
75%      0.602       1.048
max      3.215       1.427

🚀 Hari Return Tertinggi: 2023-10-13 (3.21%)
📉 Hari Return Terendah: 2024-06-07 (-3.57%)

🔗 Korelasi antar fitur:
             Return  Volatility   RSI  MACD  Signal
Return        1.00       -0.02  0.23  0.03   -0.04
Volatility   -0.02        1.00  0.07  0.18    0.23
RSI           0.23        0.07  1.00  0.72    0.50
MACD          0.03        0.18  0.72  1.00    0.94
Signal       -0.04        0.23  0.50  0.94    1.00
